In [ ]:
import numpy as np
import itertools
import json

with open("output_data_kmeans/nodes_100.json", "r", encoding="utf-8") as f:
    nodes_100_list = json.load(f)

# Giả sử đây là output từ Algorithm 1
clusters_output = {
    0: {"nodes": [0, 3, 5, 7, 10], "center": (12.3, 45.6, 30.2), "cluster_head": 5},
    1: {"nodes": [1, 2, 8, 9], "center": (80.1, 20.4, 15.7), "cluster_head": 2},
    2: {"nodes": [4, 6, 11, 12, 13], "center": (40.5, 60.0, 25.1), "cluster_head": 6}
}

# Bước 1: Trích danh sách cluster head và tọa độ nút đầu cụm
cluster_heads = [] # danh sách các nút đầu cụm
positions = {}
for cid, info in nodes_100_list.items():
    ch = info["cluster_head"]
    cluster_heads.append(ch)
    positions[ch] = np.array(info["center"])   # tọa độ tâm cụm

print("Cluster heads:", cluster_heads)

# Bước 2: Tính ma trận thời gian di chuyển giữa các nút tâm cụm
v_AUV = 10.0  # giả sử vận tốc AUV (m/s)
n = len(cluster_heads)
travel_time = np.zeros((n, n)) # ma trận thời gian di chuyển giữa các nút tâm cụm

for i in range(n):
    for j in range(n):
        if i != j:
            dist = np.linalg.norm(positions[cluster_heads[i]] - positions[cluster_heads[j]])
            travel_time[i][j] = dist / v_AUV   # tính thời gian đi từ nút i tới nút j
        else:
            travel_time[i][j] = 0

print("\nMa trận thời gian di chuyển:")

print(np.round(travel_time, 2))

# Bước 3: Thuật toán chọn đường đi (Algorithm 2)
def path_selection(cluster_heads, travel_time):
    n = len(cluster_heads)
    best_time = float('inf')
    best_path = None

    for path in itertools.permutations(range(n)):  # duyệt tất cả đường đi
        total_time = 0
        for i in range(1, n):
            total_time += travel_time[path[i-1]][path[i]]
        if total_time < best_time:
            best_time = total_time
            best_path = path
    return best_path, best_time

best_path, best_time = path_selection(cluster_heads, travel_time)

# Bước 4: Hiển thị kết quả
print("\nĐường đi AUV tối ưu (theo id của nút tâm cụm):")
print(" → ".join(str(cluster_heads[i]) for i in best_path))
print(f"Tổng thời gian di chuyển nhỏ nhất: {best_time:.2f} s")


Cluster heads: [12, 7, 14, 18, 15, 10, 4]

Ma trận thời gian di chuyển:
[[ 0.    4.55  6.86  7.84 10.18  7.24  8.97]
 [ 4.55  0.    3.78  6.85  5.97  5.79  6.96]
 [ 6.86  3.78  0.    4.31  4.65  6.34  9.05]
 [ 7.84  6.85  4.31  0.    7.27  5.97 10.24]
 [10.18  5.97  4.65  7.27  0.    6.54  7.66]
 [ 7.24  5.79  6.34  5.97  6.54  0.    4.59]
 [ 8.97  6.96  9.05 10.24  7.66  4.59  0.  ]]

Đường đi AUV tối ưu (theo id của nút tâm cụm):
4 → 10 → 18 → 14 → 15 → 7 → 12
Tổng thời gian di chuyển nhỏ nhất: 30.05 s
